# Задание 2. SQL

## 2.1 Очень усердные ученики.

## 2.1.1 Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

## 2.1.2 Задача

Дана таблица default.peas, default.studs


Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

## 2.2 Оптимизация воронки

## 2.2.1 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

## 2.2.2 Задача

Дана таблицы: default.peas (см. выше), default.studs:

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

ARPU 
ARPAU 
CR в покупку 
СR активного пользователя в покупку 
CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

2.1.2 Перед тем как работать с таблицей default.peas, посмотрим на неё

In [ ]:
SELECT 
    *
FROM 
    peas
LIMIT 100
Можно заметить, что месяца и даты слишком часто повторяются, а есть ли вообще промежуток длинной
в месяц в наших данных

In [3]:
SELECT 
    MIN(timest),  -- минимальная дата в колонке timest
    MAX(timest)   -- максимальная дата в колонке timest
FROM 
    peas
LIMIT 10

На выводе получаем:
    min(timest) 2021-10-30 13:00:00.000
    max(timest) 2021-10-31 17:29:52.000
А это значит, что данные у нас содержаться за 28,5 часов, а значит понятие усердного студента
не совсем коректно считать за месяц, посмотрим кто из студентов за это время решил >= 20 задач,
воспользовавшись простым запросом:

In [ ]:
SELECT 
    st_id,
    COUNT(correct) AS numcorrect -- выбираем какие поля будут в результирующей табл. и применим
                                    агрерирующую функцию к колонке correct, чтобы посчитать кол-во
                                    усердных студентов.
FROM 
    peas                         -- из таблицы peas. 
WHERE 
    correct = '1'                -- не забываем про условие, нам нужны, только те полльзователи
                                 -- которые правильно решили задачу, поэтому ставим условие в блок
                                 -- WHERE
GROUP BY 
    st_id                        -- группируем по студентам, чтобы напротив id студента мы видели
                                 -- количество рещенных задач
HAVING
    numcorrect >=20             -- блок HAVING отвечает за группировку в результирующей таблице,
                                -- добавляем интересующее нас условие >=20 корректно решенных задач
LIMIT 100

Остается обернуть этот запрос в подзапрос и посчитать количество усердных студентов

In [ ]:
SELECT
 COUNT(DISTINCT(student.st_id)) AS diligent_student -- считаем количество уникальных студентов и 
                                          -- переименуем поле в diligent_student DISTINCT в целом
                                          -- Необязателен т.к. есть групировка по st_id и этим мы себя
                                          -- перепроверяем и с DISTINCT и без ответ получается 
                                          -- одинаковый, ответ: 136
FROM                                      
    (
    SELECT 
        st_id,
        COUNT(correct) AS numcorrect
    FROM peas
    WHERE 
        correct = '1'                    -- можно добавить AND toStartOfMonth(timest) = '2021-10-01'
                                         -- чтобы приводить к началу месяца даты и смотреть за
                                         -- интересующий нас месяц, если данные дополнятся
    GROUP BY 
        st_id
    HAVING
        numcorrect >=20
    ) AS student                          -- имя подзапроса к которому будет обращаться
LIMIT 100

Ответ: 136 Усердных студента решили более 20 задачек в течение 28.5 часов и месяца соответсвенно (т.к. данные только за 2 дня)

2.2.2 Перед тем как работать с таблицами посмотрим на таблицы и составис предварительный план действия

In [ ]:
SELECT 
*
FROM studs
LIMIT 100

Колонки:
    1. st_id - ид клиента
    2. test_grp - группа пользователя

In [ ]:
SELECT 
*
FROM peas
LIMIT 100

Колонки:
    1. st_id - ид клиента
    2. timest - время выполнения задания
    3. correcrt - выполнено задание верно(1) или нет(0).
    4. subject - предмет

In [ ]:
SELECT 
*
FROM final_project_check
LIMIT 100

Колонки:
    1. st_id - ид клиента
    2. sale_time - время покупки
    3. money - Цена, по которой приобрели данный курс
    4. subject - предмет

Далее помощью функции WITH(структура WITH as имя_таблицы(), as имя_таблицы_2() и т.д.) создадим временные таблицы, которые будем использовать ниже.
Разбиваю чтобы проверить работоспособность кода внутри скобок.
Выбираем уникальных студентов и их принадлежность к группе.

In [ ]:
WITH 
    all AS 
(
    SELECT
        DISTINCT(st_id) AS st_id_all,
        test_grp AS grp
    FROM
        studs
),

Отбираем студентов, которые решили правильно задачи/чу из таблицы peas. Выводим их ид, предмет и сумму правильно выполненых заданий. Воспользуемся запросом из первого задания, добавив разбивку по предметам и установим фильтр на больше или равно 10 решенных заданий.

In [ ]:
more_than_10 AS
(
    SELECT 
        st_id AS st_id_more_than_10,
        subject,
        COUNT(correct) AS numcorrect
    FROM peas
    WHERE 
        correct = '1'                  
    GROUP BY 
        st_id,
        subject
    HAVING
        numcorrect > 10
),

Отбираем студентов и считаем их оплаты из таблицы final_project_check.

In [ ]:
buys AS
(
    SELECT
        st_id,
        subject,
        SUM(money) as rev_subject
    FROM 
        final_project_check
    GROUP BY 
        st_id, 
        subject
),

Добавляем ещё одну вспомогательную таблицу, в которой будем считать активных пользователей по математике и использовать ее для расчета CR по математике в результирующес запросе

In [5]:
math AS
(
    SELECT 
    st_id AS math_id,
    SUM(CASE WHEN correct = '1' AND subject = 'Math' THEN 1 ELSE 0 END) AS math_correct_quantity
    FROM 
        peas
    WHERE subject = 'Math'
    GROUP BY 
        math_id
    HAVING 
        SUM(CASE WHEN correct = '1' AND subject = 'Math' THEN 1 ELSE 0 END) >= 2 -- фильтрация 
-- после агрегации, отставляем только тех кто решил больше 2 задач по математике верно.
),

Объединим две таблицы all и тех кто активный больше 10 задач (код в SQL работать не будет т.к. это часть запроса и он ссылается на другие таблицы)

In [ ]:
SELECT
    l.st_id_all as st_id_all,
    l.grp as grp,
    r.st_id_more_than_10 as st_id_more_than_10,
    r.subject as subject
FROM
    all as l
LEFT JOIN more_than_10 as r
ON l.st_id_all = r.st_id_more_than_10


Джоиним сюда еще одну таблицу - buy добавляем код, который выше

In [2]:
SELECT 
    Levo.st_id AS st_id_buy,
    Levo.subject AS subject_buy,
    Levo.rev_subject AS rev,
    Pravo.st_id_all AS st_id_all,
    Pravo.grp AS grp,
    Pravo.st_id_more_than_10 AS st_id_more_than_10
FROM buys AS Levo
RIGHT JOIN
         (
            SELECT
                l.st_id_all as st_id_all,
                l.grp as grp,
                r.st_id_more_than_10 as st_id_more_than_10,
                r.subject as subject
            FROM
                all as l
            LEFT JOIN more_than_10 as r
            ON l.st_id_all = r.st_id_more_than_10
         ) AS Pravo
        ON Levo.st_id = Pravo.st_id_all AND  Levo.subject = Pravo.subject

Остается приджойнить только таблицу по математике и добавить вывод количества в блок SELECT, создаем результирующую таблицу 

In [7]:
full AS
(
    SELECT
        new_levo.math_id as math_id, --  количество активных пользователей по математике
        new_levo.math_correct_quantity as math_correct_quantity, -- строчку можно убрать, в результирующем 
                                                                 -- запросе сагрегирую по math_id
        new_pravo.st_id_buy as st_id_buy, -- ид студентов которые покупали
        new_pravo.rev as rev,             -- сумма покупок
        new_pravo.st_id_all as st_id_all, -- ид всеех пользователей
        new_pravo.grp as grp,             -- группы контрольная и пилотная
        new_pravo.st_id_more_than_10 as st_id_more_than_10, -- ид пользователей активных >10 действий
        new_pravo.subject_buy as subject                    -- предмет
    FROM
        math as new_levo
    RIGHT JOIN

    (
        SELECT 
            Levo.st_id AS st_id_buy,
            Levo.subject AS subject_buy,
            Levo.rev_subject AS rev,
            Pravo.st_id_all AS st_id_all,
            Pravo.grp AS grp,
            Pravo.st_id_more_than_10 AS st_id_more_than_10
        FROM buys AS Levo
        RIGHT JOIN
         (
            SELECT
                l.st_id_all as st_id_all,
                l.grp as grp,
                r.st_id_more_than_10 as st_id_more_than_10,
                r.subject as subject
            FROM
                all as l
            LEFT JOIN more_than_10 as r
            ON l.st_id_all = r.st_id_more_than_10
         ) AS Pravo
        ON Levo.st_id = Pravo.st_id_all AND  Levo.subject = Pravo.subject
    ) AS new_pravo
ON new_pravo.st_id_buy = new_levo.math_id
)

##### Финальный блок запроса, в котором мы используем данные полученные в результирующей таблице.
- ARPU Это вычисление среднего дохода на пользователя (Average Revenue Per User, ARPU). Мы суммируем доход (rev) и делим на количество уникальных пользователей (st_id_all), чтобы получить средний доход на пользователя.
- ARPAU  Это вычисление среднего дохода на активного (решающего больше 10 задач, далее будем использовать именно определение активного, активных по математике считается если он решил больше двух задач) пользователя (Average Revenue Per Active User, ARPAU). Мы суммируем доход (rev) и делим на количество уникальных решающих/активных пользователей (st_id_more_than_10), чтобы получить средний доход на активного пользователя.
- CR в покупку Это вычисление конверсии в покупку (Conversion Rate to Purchase) в процентах среди всех пользователей. Мы считаем количество уникальных платящих пользователей (st_id_buy) и делим на количество уникальных пользователей (st_id_all), затем умножаем на 100 для получения процентного значения.
- СR активного пользователя в покупку Это вычисление конверсии в покупку среди активных пользователей (Conversion Rate to Purchase Among Active Users) в процентах. Мы считаем количество уникальных платящих пользователей (st_id_buy) и делим на количество уникальных активных пользователей(st_id_more_than_10), затем умножаем на 100.
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике Это вычисление конверсии в предмете Math среди активных пользователей в процентах. Мы считаем количество активных пользователей (a_id), которые выбрали предмет Math (p_subject = 'Math' и решили больше 2х задач) и делим на общее количество активных пользователей, затем умножаем на 100.

In [ ]:
SELECT 
    grp,
    SUM(rev)/COUNT(DISTINCT(st_id_all)) AS ARPU,
    SUM(rev) / COUNT(DISTINCT(st_id_more_than_10)) AS ARPAU,
    COUNT(DISTINCT(st_id_buy)) / COUNT(DISTINCT(st_id_all)) * 100 AS CR_pokupla,
    COUNT(DISTINCT (st_id_buy)) / COUNT(DISTINCT (st_id_more_than_10)) * 100 AS CR_active_pokupka,
    COUNT(DISTINCT CASE WHEN subject = 'Math' THEN st_id_buy END) / COUNT(DISTINCT (math_id)) * 100 AS cr_math
FROM full
GROUP BY

Итоговый код

In [ ]:
WITH 
    all AS 
(
    SELECT
        DISTINCT(st_id) AS st_id_all,
        test_grp AS grp
    FROM
        studs
),

more_than_10 AS
(
    SELECT 
        st_id AS st_id_more_than_10,
        subject,
        COUNT(correct) AS numcorrect
    FROM peas
    WHERE 
        correct = '1'                  
    GROUP BY 
        st_id,
        subject
    HAVING
        numcorrect > 10
),

buys AS
(
    SELECT
        st_id,
        subject,
        SUM(money) as rev_subject
    FROM 
        final_project_check
    GROUP BY 
        st_id, 
        subject
),

math AS
(
    SELECT 
    st_id AS math_id,
    SUM(CASE WHEN correct = '1' AND subject = 'Math' THEN 1 ELSE 0 END) AS math_correct_quantity
    FROM 
        peas
    WHERE subject = 'Math'
    GROUP BY 
        math_id
    HAVING 
        SUM(CASE WHEN correct = '1' AND subject = 'Math' THEN 1 ELSE 0 END) >= 2
),


full AS
(
    SELECT
        new_levo.math_id as math_id, --  количество активных пользователей по математике
        new_levo.math_correct_quantity as math_correct_quantity, -- строчку можно убрать, в результирующем 
                                                                 -- запросе сагрегирую по math_id
        new_pravo.st_id_buy as st_id_buy, -- ид студентов которые покупали
        new_pravo.rev as rev,             -- сумма покупок
        new_pravo.st_id_all as st_id_all, -- ид всеех пользователей
        new_pravo.grp as grp,             -- группы контрольная и пилотная
        new_pravo.st_id_more_than_10 as st_id_more_than_10, -- ид пользователей активных >10 действий
        new_pravo.subject_buy as subject                    -- предмет
    FROM
        math as new_levo
    RIGHT JOIN

    (
        SELECT 
            Levo.st_id AS st_id_buy,
            Levo.subject AS subject_buy,
            Levo.rev_subject AS rev,
            Pravo.st_id_all AS st_id_all,
            Pravo.grp AS grp,
            Pravo.st_id_more_than_10 AS st_id_more_than_10
        FROM buys AS Levo
        RIGHT JOIN
         (
            SELECT
                l.st_id_all as st_id_all,
                l.grp as grp,
                r.st_id_more_than_10 as st_id_more_than_10,
                r.subject as subject
            FROM
                all as l
            LEFT JOIN more_than_10 as r
            ON l.st_id_all = r.st_id_more_than_10
         ) AS Pravo
        ON Levo.st_id = Pravo.st_id_all AND  Levo.subject = Pravo.subject
    ) AS new_pravo
ON new_pravo.st_id_buy = new_levo.math_id
)
SELECT 
    grp,
    SUM(rev)/COUNT(DISTINCT(st_id_all)) AS ARPU,
    SUM(rev) / COUNT(DISTINCT(st_id_more_than_10)) AS ARPAU,
    COUNT(DISTINCT(st_id_buy)) / COUNT(DISTINCT(st_id_all)) * 100 AS CR_pokupla,
    COUNT(DISTINCT (st_id_buy)) / COUNT(DISTINCT (st_id_more_than_10)) * 100 AS CR_active_pokupka,
    COUNT(DISTINCT CASE WHEN subject = 'Math' THEN st_id_buy END) / COUNT(DISTINCT (math_id)) * 100 AS cr_math
FROM full
GROUP BY
grp